In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import json
import pandas as pd
import os

In [3]:
deprel = "obl"

In [4]:
root = ".../margendatud/"+deprel
fil = root + "/" + "project-1-at-2023-02-09-10-44-b638f24b.json"
with open(fil, encoding="utf-8") as f:
    data = json.load(f)

In [5]:
fil2 = ".../train_dev_for_labelstudio/" + deprel + "/" + "ls_puupank_obl_export_big_v2_fileinfo.txt"
with open(fil2, "r", encoding="utf-8") as f:
    fileinfo = f.readlines()
    

In [6]:
data[0]

{'id': 1,
 'annotations': [{'id': 1,
   'completed_by': 1,
   'result': [{'value': {'start': 0,
      'end': 15,
      'text': 'Esimesel korral',
      'labels': ['obl_phrases']},
     'id': 'w4z0P5O6AS',
     'from_name': 'label',
     'to_name': 'text',
     'type': 'labels',
     'origin': 'prediction'},
    {'value': {'choices': ['bound']},
     'id': 'VANI2KmoXI',
     'from_name': 'review',
     'to_name': 'text',
     'type': 'choices',
     'origin': 'manual'}],
   'was_cancelled': False,
   'ground_truth': False,
   'created_at': '2023-02-02T13:05:58.289044Z',
   'updated_at': '2023-02-02T13:05:58.289080Z',
   'lead_time': 450.93,
   'prediction': {'id': 1,
    'model_version': 'undefined',
    'created_ago': '8\xa0minutes',
    'result': [{'to_name': 'text',
      'from_name': 'label',
      'type': 'labels',
      'value': {'start': 0,
       'end': 15,
       'text': ['Esimesel', 'korral'],
       'labels': ['obl_phrases']}}],
    'score': None,
    'cluster': None,
    'ne

In [7]:
all_data = []

for obj in data:
    idx = obj["id"]
    removed = obj["annotations"][0]["result"][0]["value"]["text"]
    if len(obj["annotations"][0]["result"]) == 1:
        print(idx)
    else:
        verdict = obj["annotations"][0]["result"][1]["value"]["choices"][0]
        sentence = obj["data"]["text"]
        
        remove_start = obj["annotations"][0]["result"][0]["value"]["start"]
        remove_end = obj["annotations"][0]["result"][0]["value"]["end"]
        short_sent = sentence[:remove_start] + sentence[remove_end:].strip()
        
        match = False
        for line in fileinfo[1:]:
            line = line.strip()
            if sentence.replace(" ", "") in line.replace(" ", "") and removed==line.split("\\")[2]:
                fname = line.split("\\")[0]
                cons_score = line.split("\\")[3]
                ual = line.split("\\")[4]
                la = line.split("\\")[5]
                root = ".../UDpuupank/UD2_11_udreposse/"
                if os.path.isfile(root+"Train/"+fname):
                    fpath = "Train/"+fname
                elif os.path.isfile(root+"Dev/"+fname):
                    fpath = "Dev/"+fname
                    
                match = True
                
                all_data.append([idx, fpath, sentence, remove_start, remove_end, removed, verdict, short_sent, cons_score, ual, la])    
                break
        
        #if not match:
        #    print(sentence)

        #all_data.append([idx, fpath, sentence, remove_start, remove_end, removed, verdict, short_sent, cons_score, ual, la])

In [8]:
df = pd.DataFrame(all_data, columns=["id", "fpath", "sentence", "remove_start", "remove_end", "removed", "type", "short_sent", "cons_score", "ual", "la"])


In [9]:
df

,id,fpath,sentence,remove_start,remove_end,removed,type,short_sent,cons_score,ual,la
0,1,Dev/aja_luup200009_ud211.conllu,"Esimesel korral oli vene rahvas sõja vastu , a...",0,15,Esimesel korral,bound,"oli vene rahvas sõja vastu , aga seekord ollak...",100.0,100.0,100.0
1,2,Train/aja_ee199920_osa_1_ud211.conllu,Suusakeskus hakkas poolest talvest tööle ja mu...,19,34,poolest talvest,free,Suusakeskus hakkas tööle ja muutus üsna ruttu ...,100.0,100.0,100.0
2,3,Train/aja_ee199920_osa_1_ud211.conllu,Suusakeskus hakkas poolest talvest tööle ja mu...,35,40,tööle,bound,Suusakeskus hakkas poolest talvest ja muutus ü...,90.0,90.0,90.0
3,4,Train/tea_AA_05_6_ud211.conllu,Hemodialüüsis patsientide jälgimisel jääb maha...,0,36,Hemodialüüsis patsientide jälgimisel,unnatural,jääb maha tohutu suur hulk meditsiinilist info...,89.5,89.5,89.5
4,5,Train/tea_AA_05_6_ud211.conllu,Hemodialüüsis patsientide jälgimisel jääb maha...,96,104,arstidel,unnatural,Hemodialüüsis patsientide jälgimisel jääb maha...,90.5,90.5,90.5
...,...,...,...,...,...,...,...,...,...,...,...
2739,2746,Train/aja_sloleht20071217_ud211.conllu,Iraagi kurdi armee juhtkond teatas sõdurite sa...,35,115,"sõdurite saatmisest Türgi piiri äärde "" uurima...",bound,Iraagi kurdi armee juhtkond teatas .,66.7,83.3,83.3
2740,2747,Train/aja_ee200110_osa_4_ud211.conllu,Peale Väike-Maarja mõõdetakse Eestis lund veel...,0,18,Peale Väike-Maarja,unnatural,mõõdetakse Eestis lund veel rohkem kui poolesa...,88.9,88.9,100.0
2741,2748,Train/aja_ee200110_osa_4_ud211.conllu,Peale Väike-Maarja mõõdetakse Eestis lund veel...,30,36,Eestis,free,Peale Väike-Maarja mõõdetakse lund veel rohkem...,90.0,90.0,100.0
2742,2749,Train/tea_eesti_arst_2004_osa_2_ud211.conllu,See uuring aitab amüloidoosi diagnoosimisele l...,17,51,amüloidoosi diagnoosimisele lisaks,free,See uuring aitab hinnata amüloidi hulka ja hai...,None,None,None


In [10]:
df_free = df.loc[df['type'] == "free"]
df_free

,id,fpath,sentence,remove_start,remove_end,removed,type,short_sent,cons_score,ual,la
1,2,Train/aja_ee199920_osa_1_ud211.conllu,Suusakeskus hakkas poolest talvest tööle ja mu...,19,34,poolest talvest,free,Suusakeskus hakkas tööle ja muutus üsna ruttu ...,100.0,100.0,100.0
5,6,Train/aja_ee200110_osa_4_ud211.conllu,Äkki ongi preili Mikomägi maailma kõige seksik...,26,33,maailma,free,Äkki ongi preili Mikomägi kõige seksikam eluso...,100.0,100.0,100.0
6,7,Train/ilu_valton_ud211.conllu,"Lehmal sündis vasikas , igavene suur , aga leh...",0,6,Lehmal,free,"sündis vasikas , igavene suur , aga lehm ei ka...",92.9,92.9,100.0
7,8,Train/aja_ee199920_osa_2_ud211.conllu,Ivo ise otsustas eriüksusest ära tulla aasta e...,17,28,eriüksusest,free,Ivo ise otsustas ära tulla aasta eest .,87.5,87.5,100.0
8,9,Train/aja_ee199920_osa_2_ud211.conllu,Ivo ise otsustas eriüksusest ära tulla aasta e...,39,49,aasta eest,free,Ivo ise otsustas eriüksusest ära tulla .,100.0,100.0,100.0
...,...,...,...,...,...,...,...,...,...,...,...
2729,2736,Train/aja_pm20001004_osa_1_ud211.conllu,Tartu majaomanikud ei pääse ka sel sügisel leh...,110,116,talvel,free,Tartu majaomanikud ei pääse ka sel sügisel leh...,73.3,76.7,83.3
2731,2738,Train/aja_pm20001004_osa_1_ud211.conllu,Tartu majaomanikud ei pääse ka sel sügisel leh...,134,152,linna kõnniteedelt,free,Tartu majaomanikud ei pääse ka sel sügisel leh...,72.4,75.9,86.2
2738,2745,Train/ilu_remsu_ud211.conllu,Euroopasse jõudis Hiina siid 2. sajandil päras...,115,125,Euroopasse,free,Euroopasse jõudis Hiina siid 2. sajandil päras...,91.3,91.3,95.7
2741,2748,Train/aja_ee200110_osa_4_ud211.conllu,Peale Väike-Maarja mõõdetakse Eestis lund veel...,30,36,Eestis,free,Peale Väike-Maarja mõõdetakse lund veel rohkem...,90.0,90.0,100.0


In [11]:
df_free.to_csv(f"deprel_free_analysis/csv_files/{deprel}_free.csv", sep=";", index=False)

In [12]:
df_free_cons = df_free[df_free["cons_score"]=="100.0"]
df_free_cons

,id,fpath,sentence,remove_start,remove_end,removed,type,short_sent,cons_score,ual,la
1,2,Train/aja_ee199920_osa_1_ud211.conllu,Suusakeskus hakkas poolest talvest tööle ja mu...,19,34,poolest talvest,free,Suusakeskus hakkas tööle ja muutus üsna ruttu ...,100.0,100.0,100.0
5,6,Train/aja_ee200110_osa_4_ud211.conllu,Äkki ongi preili Mikomägi maailma kõige seksik...,26,33,maailma,free,Äkki ongi preili Mikomägi kõige seksikam eluso...,100.0,100.0,100.0
8,9,Train/aja_ee199920_osa_2_ud211.conllu,Ivo ise otsustas eriüksusest ära tulla aasta e...,39,49,aasta eest,free,Ivo ise otsustas eriüksusest ära tulla .,100.0,100.0,100.0
18,19,Train/arborest_ud211.conllu,Peeter jooksis toast koridori ust avama .,15,20,toast,free,Peeter jooksis koridori ust avama .,100.0,100.0,100.0
29,30,Train/aja_sloleht20071217_ud211.conllu,USA teadlased peavad pikaealisuse puhul head f...,61,71,saledusest,free,USA teadlased peavad pikaealisuse puhul head f...,100.0,100.0,100.0
...,...,...,...,...,...,...,...,...,...,...,...
2697,2704,Train/tea_eesti_arst_2004_osa_3_ud211.conllu,Hüvitisi on iseloomustatud mitmetes Euroopa ko...,64,85,määrusetõlgendamiseks,free,Hüvitisi on iseloomustatud mitmetes Euroopa ko...,100.0,100.0,100.0
2710,2717,Train/aja_pm20001004_osa_2_ud211.conllu,Edasi on nad paigutunud ebaproportsionaalselt ...,46,85,erinevatesse sotsiaalsetesse kihtidesse,free,Edasi on nad paigutunud ebaproportsionaalselt .,100.0,100.0,100.0
2716,2723,Train/aja_pm20001004_osa_2_ud211.conllu,1710. aastal lahkus ülikool pealetungivate Ven...,28,59,pealetungivate Vene vägede eest,free,1710. aastal lahkus ülikool Rootsi .,100.0,100.0,100.0
2717,2724,Train/aja_pm20001004_osa_2_ud211.conllu,1710. aastal lahkus ülikool pealetungivate Ven...,60,66,Rootsi,free,1710. aastal lahkus ülikool pealetungivate Ven...,100.0,100.0,100.0


In [13]:
df_free_cons.to_csv(f"deprel_free_analysis/csv_files/{deprel}_free_conserved.csv", sep=";", index=False)

In [10]:
df_bound = df.loc[df['type'] == "bound"]
df_bound

,id,fpath,sentence,remove_start,remove_end,removed,type,short_sent,cons_score,ual,la
0,1,Dev/aja_luup200009_ud211.conllu,"Esimesel korral oli vene rahvas sõja vastu , a...",0,15,Esimesel korral,bound,"oli vene rahvas sõja vastu , aga seekord ollak...",100.0,100.0,100.0
2,3,Train/aja_ee199920_osa_1_ud211.conllu,Suusakeskus hakkas poolest talvest tööle ja mu...,35,40,tööle,bound,Suusakeskus hakkas poolest talvest ja muutus ü...,90.0,90.0,90.0
10,11,Train/tea_geofyysika.conllu,Selles kunstis ei saanud nähtavasti keegi Aris...,42,200,"Aristotelese ( 384-322 e. Kr. ) vastu , kes os...",bound,Selles kunstis ei saanud nähtavasti keegi .,100.0,100.0,100.0
16,17,Train/aja_luup200202_osa_3_ud211.conllu,"Erastamise lõppedes selgus , et insaiderite kä...",0,10,Erastamise,bound,"lõppedes selgus , et insaiderite kätte oli lib...",92.9,92.9,100.0
17,18,Train/aja_luup200202_osa_3_ud211.conllu,"Erastamise lõppedes selgus , et insaiderite kä...",32,49,insaiderite kätte,bound,"Erastamise lõppedes selgus , et oli libisenud ...",92.3,92.3,100.0
...,...,...,...,...,...,...,...,...,...,...,...
2732,2739,Train/aja_ml200247_osa_4_ud211.conllu,"Siis , kui taheti jõuga haldusterritoriaalset ...",18,23,jõuga,bound,"Siis , kui taheti haldusterritoriaalset reform...",100.0,100.0,100.0
2733,2740,Train/aja_ee200110_osa_5_ud211.conllu,"Ühel keskealisel mehel , kes ajab kokku varand...",61,73,lasteaedades,bound,"Ühel keskealisel mehel , kes ajab kokku varand...",100.0,100.0,100.0
2736,2743,Train/ilu_remsu_ud211.conllu,Euroopasse jõudis Hiina siid 2. sajandil päras...,64,84,veel mõnisada aastat,bound,Euroopasse jõudis Hiina siid 2. sajandil päras...,90.5,90.5,95.2
2739,2746,Train/aja_sloleht20071217_ud211.conllu,Iraagi kurdi armee juhtkond teatas sõdurite sa...,35,115,"sõdurite saatmisest Türgi piiri äärde "" uurima...",bound,Iraagi kurdi armee juhtkond teatas .,66.7,83.3,83.3


In [11]:
df_bound.to_csv(f"deprel_free_analysis/csv_files/{deprel}_bound.csv", sep=";", index=False)

In [12]:
df_comma = df.loc[df['type'] == "redundant comma"]
df_comma

,id,fpath,sentence,remove_start,remove_end,removed,type,short_sent,cons_score,ual,la
439,440,Train/aja_ee200110_osa_2_ud211.conllu,"Päev hiljem , reede õhtul pidasid Tallinna Põh...",0,11,Päev hiljem,redundant comma,", reede õhtul pidasid Tallinna Põhja politseio...",88.9,88.9,100.0
440,441,Train/aja_ee200110_osa_2_ud211.conllu,"Päev hiljem , reede õhtul pidasid Tallinna Põh...",14,25,reede õhtul,redundant comma,"Päev hiljem , pidasid Tallinna Põhja politseio...",77.8,77.8,100.0
496,497,Train/tea_toohoive.conllu,"Selleks , et hinnata keskmise palga ja miinimu...",0,67,"Selleks , et hinnata keskmise palga ja miinimu...",redundant comma,", tuleb vaadata ka seda , mis on juhtunud meie...",100.0,100.0,100.0
515,516,Train/ilu_remsu_ud211.conllu,"Eesti keeles ja meeleski on juurdunud , tõsi k...",40,44,tõsi,redundant comma,"Eesti keeles ja meeleski on juurdunud , küll m...",96.3,96.3,100.0
577,578,Train/aja_pm19980926e_ud211.conllu,2000 aastat e.m.a - Egiptuse hauakambritest on...,0,17,2000 aastat e.m.a,redundant comma,- Egiptuse hauakambritest on leitud joonistusi...,77.8,77.8,88.9
696,697,Train/ilu_kanep_ud211.conllu,"Tol hommikul , kui sa vihaselt poodi läksid , ...",0,43,"Tol hommikul , kui sa vihaselt poodi läksid",redundant comma,", olin õnnetust intsidendist juba kuulnud .",100.0,100.0,100.0
733,734,Dev/aja_luup200009_ud211.conllu,"Teiste sõnadega , kui Kuu oleks Maast 10 korda...",0,15,Teiste sõnadega,redundant comma,", kui Kuu oleks Maast 10 korda kaugemal kui pr...",100.0,100.0,100.0
829,830,Train/aja_luup200106_osa_3_ud211.conllu,"Kõige halvem on , kui noorele , kes väga tahab...",22,116,"noorele , kes väga tahab akadeemilist haridust...",redundant comma,"Kõige halvem on , kui , öeldakse , sul pole mu...",86.7,86.7,100.0
931,932,Dev/tea_eesti_arst_2004_osa_1_ud211.conllu,"Haigetele , kellel pole võimalik viibida pikem...",0,59,"Haigetele , kellel pole võimalik viibida pikem...",redundant comma,", võib siiski julgelt soovitada ka kuuepäevast...",88.9,88.9,100.0
959,960,Train/aja_pm20001004_osa_2_ud211.conllu,"Rootsi-Vene sõja ajal ( 1656-1661 ) , kui ülik...",0,35,Rootsi-Vene sõja ajal ( 1656-1661 ),redundant comma,", kui ülikoolis katkes õppetöö , olid ülikooli...",94.7,94.7,94.7


In [13]:
df_comma.to_csv(f"deprel_free_analysis/csv_files/{deprel}_redundant_comma.csv", sep=";", index=False)

In [14]:
df_un = df.loc[df['type'] == "unnatural"]
df_un

,id,fpath,sentence,remove_start,remove_end,removed,type,short_sent,cons_score,ual,la
3,4,Train/tea_AA_05_6_ud211.conllu,Hemodialüüsis patsientide jälgimisel jääb maha...,0,36,Hemodialüüsis patsientide jälgimisel,unnatural,jääb maha tohutu suur hulk meditsiinilist info...,89.5,89.5,89.5
4,5,Train/tea_AA_05_6_ud211.conllu,Hemodialüüsis patsientide jälgimisel jääb maha...,96,104,arstidel,unnatural,Hemodialüüsis patsientide jälgimisel jääb maha...,90.5,90.5,90.5
9,10,Train/tea_geofyysika.conllu,Selles kunstis ei saanud nähtavasti keegi Aris...,0,14,Selles kunstis,unnatural,ei saanud nähtavasti keegi Aristotelese ( 384-...,12.9,22.6,16.1
12,13,Train/tea_AA_05_6_ud211.conllu,Igal ajahetkel toimub vaadeldavas süsteemis va...,0,14,Igal ajahetkel,unnatural,toimub vaadeldavas süsteemis vaid ühe andmelõi...,100.0,100.0,100.0
14,15,Train/tea_dr8047.conllu,Huvitava faktina märkasid J. O. Hardin jt. ( 1...,0,16,Huvitava faktina,unnatural,märkasid J. O. Hardin jt. ( 1964 ) küll tähtsu...,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...
2723,2730,Train/aja_sloleht20071217_ud211.conllu,"6. augustil 1997 ilmus ühes ajalehes foto , ku...",0,16,6. augustil 1997,unnatural,"ilmus ühes ajalehes foto , kus Dodi ja Diana s...",81.8,100.0,81.8
2734,2741,Train/ilu_remsu_ud211.conllu,Euroopasse jõudis Hiina siid 2. sajandil päras...,0,10,Euroopasse,unnatural,jõudis Hiina siid 2. sajandil pärast Kristust ...,91.3,91.3,95.7
2735,2742,Train/ilu_remsu_ud211.conllu,Euroopasse jõudis Hiina siid 2. sajandil päras...,29,56,2. sajandil pärast Kristust,unnatural,"Euroopasse jõudis Hiina siid , läks veel mõnis...",95.0,95.0,100.0
2737,2744,Train/ilu_remsu_ud211.conllu,Euroopasse jõudis Hiina siid 2. sajandil päras...,90,106,araablaste kaudu,unnatural,Euroopasse jõudis Hiina siid 2. sajandil päras...,86.4,86.4,95.5


In [15]:
df_un.to_csv(f"deprel_free_analysis/csv_files/{deprel}_unnatural.csv", sep=";", index=False)

In [16]:
df_dub = df.loc[df['type'] == "dubious"]
df_dub

,id,fpath,sentence,remove_start,remove_end,removed,type,short_sent,cons_score,ual,la
53,54,Train/tea_eesti_arst_2004_osa_4_ud211.conllu,Viisteist sekundit enne igat mõõtmist teatati ...,19,37,enne igat mõõtmist,dubious,Viisteist sekundit teatati uuritavale eesseisv...,85.7,85.7,85.7
127,128,Train/aja_ee199920_osa_1_ud211.conllu,""" Ühel päeval leidsin oma toast külmkapi , see...",22,31,oma toast,dubious,""" Ühel päeval leidsin külmkapi , see oli väga ...",100.0,100.0,100.0
231,232,Train/ilu_kanep_ud211.conllu,Ma ise põhimõtteliselt hea meelega väldiksin s...,255,264,sul endal,dubious,Ma ise põhimõtteliselt hea meelega väldiksin s...,88.9,93.3,95.6
367,368,Train/ilu_kivirahk_ud211.conllu,"Ärkaksin üles hauast ma taas ! """,14,20,hauast,dubious,"Ärkaksin üles ma taas ! """,100.0,100.0,100.0
552,553,Train/aja_ee200110_osa_2_ud211.conllu,Ülalkirjeldatud meditsiiniline protseduur leia...,88,99,teisipäeval,dubious,Ülalkirjeldatud meditsiiniline protseduur leia...,100.0,100.0,100.0
672,673,Train/aja_ee200110_osa_1_ud211.conllu,"Lugu on ju nii , et ühiskonnale kasutoova proj...",20,31,ühiskonnale,dubious,"Lugu on ju nii , et kasutoova projekti algatav...",71.4,71.4,85.7
756,757,Train/tea_AA_05_6_ud211.conllu,proovida kasu lõigata väga lähedase valdkonna ...,22,58,väga lähedase valdkonna populaarsuse,dubious,proovida kasu lõigata kasvust,100.0,100.0,100.0
843,844,Dev/aja_ml200247_osa6_ud211.conllu,"1990. aastal siirdus Gerda , kes vanemate ja p...",33,56,vanemate ja passi järgi,dubious,"1990. aastal siirdus Gerda , kes on soomlane ,...",92.3,92.3,100.0
863,864,Train/aja_ee200110_osa_10_ud211.conllu,Aga me oleme sellest nüüd juba nii kaugel - ja...,13,20,sellest,dubious,Aga me oleme nüüd juba nii kaugel - ja hetkel ...,92.3,92.3,100.0
1028,1029,Train/ilu_valton_ud211.conllu,"Kui vedruvõim oli lõpuni maha käinud , seisis ...",18,24,lõpuni,dubious,"Kui vedruvõim oli maha käinud , seisis põranda...",100.0,100.0,100.0


In [17]:
df_dub.to_csv(f"deprel_free_analysis/csv_files/{deprel}_dubious.csv", sep=";", index=False)

In [18]:
df_red = df.loc[df['type'] == "other redundant punctuation"]
df_red

,id,fpath,sentence,remove_start,remove_end,removed,type,short_sent,cons_score,ual,la


In [19]:
df_red.to_csv(f"deprel_free_analysis/csv_files/{deprel}_redundant_punct.csv", sep=";", index=False)

In [ ]:
df_bound = df.loc[df['type'] == "bound"]
df_bound

df_bound.to_csv(f"deprel_free_analysis/csv_files/{deprel}_bound.csv", sep=";", index=False)

df_comma = df.loc[df['type'] == "redundant comma"]
df_comma

df_comma.to_csv(f"deprel_free_analysis/csv_files/{deprel}_redundant_comma.csv", sep=";", index=False)

df_un = df.loc[df['type'] == "unnatural"]
df_un

df_un.to_csv(f"deprel_free_analysis/csv_files/{deprel}_unnatural.csv", sep=";", index=False)

df_dub = df.loc[df['type'] == "dubious"]
df_dub

df_dub.to_csv(f"deprel_free_analysis/csv_files/{deprel}_dubious.csv", sep=";", index=False)

df_red = df.loc[df['type'] == "other redundant punctuation"]
df_red

df_red.to_csv(f"deprel_free_analysis/csv_files/{deprel}_redundant_punct.csv", sep=";", index=False)

## Vabade laienditega laused conll formaadis failidesse

In [11]:
from estnltk.converters.conll.conll_exporter import sentence_to_conll
from estnltk import Text
from estnltk.taggers import WhiteSpaceTokensTagger
from estnltk.taggers import PretokenizedTextCompoundTokensTagger
from estnltk_neural.taggers.syntax.stanza_tagger.stanza_tagger import StanzaSyntaxTagger

2023-06-14 08:35:17.664711: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-06-14 08:35:18.051265: I tensorflow/core/util/port.cc:104] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-06-14 08:35:19.329466: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2023-06-14 08:35:19.329585: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] 

In [12]:
model_path=".../lib/python3.9/site-packages/estnltk_neural/taggers/syntax/stanza_tagger/stanza_resources/"
input_type="morph_extended"
stanza_tagger = StanzaSyntaxTagger(input_type=input_type, input_morph_layer=input_type, add_parent_and_children=True, resources_path=model_path)

In [13]:
tokens_tagger = WhiteSpaceTokensTagger()
compound_tokens_tagger = PretokenizedTextCompoundTokensTagger()

In [14]:
from tqdm import tqdm

In [15]:
shortened_sents_data_dev = []
original_sents_data_dev = []
shortened_sents_data_train = []
original_sents_data_train = []

for row in tqdm(df_free.iterrows()):

    file = row[1][1]
    
    folder = file.split("/")[0]
    #print(folder)
    #break
    
    with open(f".../UDpuupank/UD2_11_udreposse/{file}") as f:
        conll = f.readlines()
        
    removed = row[1][5].split(" ")
    remove_start = row[1][3]
    remove_end = row[1][4]
    original_sent = row[1][2]
    sent = row[1][2].replace(" ", "")

    important_data = []

    # collect rows related to this sentence
    start = False
    for i, line in enumerate(conll):
        line = line.strip()
        mod_line = line.replace(" ", "")
        if start and not line.startswith("#"):
            important_data.append(line)
        elif start and (line=="" or line.startswith("#")):
            break    
        elif line.startswith("#") and sent in mod_line:
            start = True
            important_data.append(conll[i-1].strip())
            important_data.append(line)
            
            
    
                
            
    
    # find lines with removed words
    if len(important_data) != 0:
        
        #if "sent_id" in important_data[0]:
        #    sent_id = important_data[0].split("=")[1].strip()
        #    if sent_id in sent_dict.keys():
        if folder == "Train":
            original_sents_data_train.append(important_data)    
        elif folder == "Dev":
            original_sents_data_dev.append(important_data)
        else:
            print(f"folder {file} problem")
            break
        
        
        short_data_train = []
        short_data_dev = []
        
        short_data_pre = []
        cut_word_ids = []
        short_data_pre.append(important_data[0].strip())

        remove_count = 0
        for line in important_data[2:]:
            #print(line)

            checked = False  
            if line != "" and "sent_id =" not in line and "text =" not in line:
                added = len(line.split("\t")[1].strip() + " ")
                
                if line.split("\t")[1] in removed:
                    if len(cut_word_ids)==0 and remove_count == remove_start:
                        cut_word_ids.append(line.split("\t")[0])
                    elif len(cut_word_ids)!=0 and remove_count <= remove_end:
                        cut_word_ids.append(line.split("\t")[0])
                    checked = True
                remove_count += added
            #if not checked:    
            #    short_data.append(line+"\n")

        short_sentence = original_sent[:remove_start] + original_sent[remove_end+1:]

        txt = Text(short_sentence)
        
        tokens_tagger.tag(txt)
        compound_tokens_tagger.tag(txt)
        txt.tag_layer("morph_extended")
        stanza_tagger.tag(txt)
        res = sentence_to_conll(txt.sentences[0], txt["stanza_syntax"], udpipe=True)
        res2 = res.split("\n")
        
        
        short_data_pre.append("# cut_word_ids = " + ",".join(cut_word_ids))
        short_data_pre.append('# ud_version = 2.11')
        short_sent_text = "# text = " + " ".join([span.text for span in txt.words])
        short_data_pre.append(short_sent_text)
        
        
        #if "sent_id" in important_data[0]:
        #    sent_id = important_data[0].split("=")[1].strip()
        #    if sent_id in sent_dict.keys():
        if folder == "Train":
            short_data_train = short_data_pre + res2
            shortened_sents_data_train += [short_data_train]    
        elif folder == "Dev":
            short_data_dev = short_data_pre + res2
            shortened_sents_data_dev += [short_data_dev]
        
    #print(original_sent)
    #print(important_data)
    #print("\n",short_sent_text)
    #print(sentence_to_conll(row[1][7][:-2] + row[1][7][-2:].strip()))
    
    #print(short_data)
    
    #break
    
    
    #else:
    #    print("PROBLEEMIGA:", row[1][2])
    
#shortened_sents_data





1199it [02:35,  7.71it/s]


In [16]:
sents_to_file = 2000

left = len(shortened_sents_data_dev) % sents_to_file
files = int(len(shortened_sents_data_dev)/sents_to_file)
if left != 0:
    files += 1

for i in range(files):
    with open(f"deprel_free_analysis/Dev/{deprel}_cut.conllu", "w", encoding="utf-8") as f:
        for obj in shortened_sents_data_dev[i*sents_to_file:(i+1)*sents_to_file]:
            for line in obj:
                f.write(line)
                f.write("\n")
                
    with open(f"deprel_free_analysis/Dev/{deprel}_original.conllu", "w", encoding="utf-8") as f:
        for obj in original_sents_data_dev[i*sents_to_file:(i+1)*sents_to_file]:
            for line in obj:
                f.write(line)
                f.write("\n")
                
                
left = len(shortened_sents_data_train) % sents_to_file
files = int(len(shortened_sents_data_train)/sents_to_file)
if left != 0:
    files += 1                
for i in range(files):
    with open(f"deprel_free_analysis/Train/{deprel}_cut.conllu", "w", encoding="utf-8") as f:
        for obj in shortened_sents_data_train[i*sents_to_file:(i+1)*sents_to_file]:
            for line in obj:
                f.write(line)
                f.write("\n")
                
    with open(f"deprel_free_analysis/Train/{deprel}_original.conllu", "w", encoding="utf-8") as f:
        for obj in original_sents_data_train[i*sents_to_file:(i+1)*sents_to_file]:
            for line in obj:
                f.write(line)
                f.write("\n")